In [61]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Задані тексти
texts_true = [
    "Після релізу додатку, наша компанія отримала дуже позитивну оцінку.",
    "Користувачі були задоволені."
    "Цей сервіс отримав ще один позитивний відгук."
]

texts_false = [
    "Компанія отримала негативний досвід.",
    "Додаток не досягнув очікуваного успіху.",
    "Оцінка компанії була негативна."
]

# Об'єднання текстів
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True, activation='relu'))  
model.add(LSTM(64, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.summary()


model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=RMSprop(learning_rate=0.001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)


2 3 5
[('після', 1), ('релізу', 1), ('додатку', 1), ('наша', 1), ('компанія', 2), ('отримала', 2), ('дуже', 1), ('позитивну', 1), ('оцінку', 1), ('користувачі', 1)]
Після релізу додатку, наша компанія отримала дуже позитивну оцінку.
[[ 0  3  4  5  6  1  2  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  0  0  0  0  0  1  2 20 21]
 [ 0  0  0  0  0 22 23 24 25 26]
 [ 0  0  0  0  0  0 27 28 29 30]]
[('компанія', 1), ('отримала', 2), ('після', 3), ('релізу', 4), ('додатку', 5), ('наша', 6), ('дуже', 7), ('позитивну', 8), ('оцінку', 9), ('користувачі', 10), ('були', 11), ('задоволені', 12), ('цей', 13), ('сервіс', 14), ('отримав', 15), ('ще', 16), ('один', 17), ('позитивний', 18), ('відгук', 19), ('негативний', 20), ('досвід', 21), ('додаток', 22), ('не', 23), ('досягнув', 24), ('очікуваного', 25), ('успіху', 26), ('оцінка', 27), ('компанії', 28), ('була', 29), ('негативна', 30)]
(5, 10) (5, 2)
Model: "sequential_7"
_________________________________________________________________
 Layer (t

In [62]:
t = "Наша компанія отримала позитивну новину.".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

res = model.predict(data_pad)

['наша', 'компанія', 'отримала', 'позитивну']
1/1 [==============================] - 0s 370ms/step


In [67]:
# Визначення результату
if np.argmax(res) == 1:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст позитивний.


In [64]:
t1 = "Досвід був негативний".lower()
data1 = tokenizer.texts_to_sequences([t1])
data_pad1 = pad_sequences(data1, maxlen=max_text_len)
print( sequence_to_text(data1[0]) )

res1 = model.predict(data_pad1)

['досвід', 'негативний']
1/1 [==============================] - 0s 16ms/step


In [68]:
# Визначення результату
if np.argmax(res1) == 0:
    print("Результат. Текст позитивний.")
else:
    print("Результат. Текст негативний.")

Результат. Текст позитивний.


**Висновок:** на лабораторній роботі я навчився створювати архітектуру рекурентної нейронної мережі для семантичного аналізу з використанням моделі LSTM.